In [1]:
#du/dx - cos(x) = 0

import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import math

rng = np.random.default_rng(seed=20)


In [2]:
n_wires=6
n_layers=3
load_filename='t1.npy'
save_filename='t1_prime.npy'
dev = qml.device('default.qubit', wires=n_wires)
loaded_params = np.load(load_filename)
print("Loaded parameters:", loaded_params)

shape = qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=n_wires)
parameters_global = rng.random(size=shape)

Loaded parameters: [[[ 7.92584000e-01  4.96577578e-01  8.37177042e-01]
  [ 1.38276104e+00  3.17091963e-01  1.31062160e+00]
  [ 6.88898411e-01  1.58963135e+00  1.35568209e+00]
  [ 1.20298708e+00 -3.08182231e-02 -2.13950015e-01]
  [ 2.00370205e-01  1.40889514e+00  7.17912895e-01]
  [ 1.19105137e+00 -2.44736755e-01  7.34317968e-01]]

 [[ 9.90709265e-01  1.73111362e-01  9.10081104e-01]
  [ 2.42752426e-01  2.53181926e-01  8.80110991e-01]
  [ 2.78517236e-02 -1.73272272e-01  1.34624995e+00]
  [ 1.58273495e+00  2.13853515e-01  1.65867505e-01]
  [-7.87294873e-01  4.30168445e-01  8.16436361e-01]
  [ 1.10474445e+00  9.51624167e-02  3.96605144e-01]]

 [[ 6.50901271e-01  5.22517705e-01  3.57765485e-01]
  [ 6.69503303e-01  6.10103196e-01  5.11909830e-01]
  [ 9.34240159e-01  1.55753642e-01  6.27905706e-01]
  [ 9.44619540e-01  6.07381587e-04  6.43282023e-01]
  [ 7.82369448e-01  6.54766215e-01  6.07836611e-01]
  [-2.45987576e-02  3.99719470e-01  4.65603022e-01]]

 [[ 3.48967112e-01  3.45820436e-01  9.9

In [3]:
def phi(x, j):  # Chebyshev
    return 2 * j * np.arccos(x)

def dphi1(x,j):
    return 2*j*(-1 / np.sqrt(1 - x ** 2))

def dphi2(x,j):
    return 2*j*(-x / ((1 - x ** 2) ** (3 / 2)))

In [4]:
@qml.qnode(dev)
def u_x(x):
    global parameters_global
    # feature map
    for j in range(n_wires):
        qml.RY(phi=phi(x,j+1),wires=j)
    # ansatz
    qml.StronglyEntanglingLayers(weights=parameters_global, wires=range(dev.num_wires)) 
    # Cost function <Z>
    return qml.expval(qml.PauliZ(0))

# shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=n_wires)
# w = rng.random(size=shape)
# print(qml.draw(u_x)(0.1,w))

In [5]:



@qml.qnode(dev, interface="autograd", diff_method="parameter-shift")
def du_x_helper(phi_x_vector):
    global parameters_global
    # feature map
    for j in range(n_wires):
        qml.RY(phi_x_vector[j],wires=j)
    # ansatz
    qml.StronglyEntanglingLayers(weights=parameters_global, wires=range(dev.num_wires)) 
    # Cost function <Z>
    return qml.expval(qml.PauliZ(0))

         
def du_x(x):
    global parameters_global
    phi_x=['Nan']*n_wires
    dphi_x=['Nan']*n_wires
    for j in range(n_wires):
        phi_x[j] = phi(x,j)
        dphi_x[j] = dphi1(x,j)
    phi_x = np.array(phi_x,requires_grad=True)
    # dphi_x = np.array(dphi_x)
    # parameters_nograd = np.array(parameters,requires_grad=False)

    grads=qml.jacobian(du_x_helper)(phi_x)   
    # print(grads)
    # raise Excepti
    grad_dphi = grads*dphi_x 
    return 0.5*np.sum(grad_dphi)

        
# shape = qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_wires)#TODO:// dynamic
# params = rng.random(size=shape)
# sample_x = np.linspace(0.1, 0.9, 10, endpoint=True)
# for x in sample_x:
#     print(du_x(x,parameters=params))

# raise Exception()
# shape = qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_wires)#TODO:// dynamic
# params = rng.random(size=shape)
# print(params)
# print(qml.grad(du_x)(0.1,params))
# raise Exception('haha')

In [6]:
sample_x = np.linspace(0.1, 0.9, 10, endpoint=True)
sample_x

tensor([0.1       , 0.18888889, 0.27777778, 0.36666667, 0.45555556,
        0.54444444, 0.63333333, 0.72222222, 0.81111111, 0.9       ], requires_grad=True)

In [7]:
def calculate_loss_boundry(parameters,err="MSE"):
    #boundry for u(x)=sin(x), u(0)=0, u(pi/2)=1
    x = [0.1,0.9]
    u = [0.1,0.78]
    u_predict=[0,0]
    error = None
    for i in range(len(x)):
        u_predict[i]=u_x(x=x[i])

    if err=="MSE":
        #MSE
        error = np.mean((np.array(u_predict) - np.array(u))**2)
    else:
        raise Exception('Not implemented yet')
    return error
# calculate_loss_boundry(weights)

In [8]:
np.cos(sample_x)

tensor([0.99500417, 0.98221347, 0.96166719, 0.93352755, 0.89801674,
        0.85541517, 0.80605921, 0.75033857, 0.68869324, 0.62160997], requires_grad=True)

In [9]:
def calculate_loss_interior(parameters,err="MSE"):
    # sample_ux = np.sin(sample_x)
    du_x_res=np.array([0.0]*len(sample_x))
    rhs = np.zeros(len(sample_x))
    error = None
    for i in range(len(sample_x)):
        du_x_res[i]=du_x(x=sample_x[i])
    lhs = du_x_res-np.cos(sample_x)
    if err=="MSE":
        #MSE
        error = np.mean((lhs - rhs)**2)
    else:
        raise Exception('Not implemented yet')
    return error

# shape = qml.StronglyEntanglingLayers.shape(n_layers=2, n_wires=n_wires)#TODO:// dynamic
# params = rng.random(size=shape)
# calculate_loss_interior(params)

In [10]:


def loss_func(params):
    global parameters_global
    loss= 0.0*calculate_loss_boundry(parameters_global)+1*calculate_loss_interior(parameters_global)
    print(loss)
    return loss

In [11]:
def plot_u_predict(params):
    
    sample_u = np.sin(sample_x)
    sample_u_predict = []
    for x in sample_x:
        sample_u_predict.append(u_x(x))  # Assuming u_x is defined elsewhere

    plt.plot(sample_x, sample_u, 'bo-', label='Actual')
    plt.plot(sample_x, sample_u_predict, 'r*-', label='Predicted')
    plt.xlabel('x')
    plt.ylabel('sin(x)')
    plt.title('Plot of sin(x)')
    plt.legend()
    plt.grid(True)
    plt.show()
# plot_u_predict()

In [12]:
from scipy.optimize import minimize

def callback(xk):
    print("Objective function value:", loss_func(xk))
    
# Minimize the objective function with the callback function
result = minimize(loss_func, parameters_global, callback=callback)

# Print the optimized result
print("Optimized value:", result.x)
print("Optimized function value:", result.fun)

/var/folders/h3/43y0n8w9269f882crb4hn2sc0000gn/T/ipykernel_35028/1436502121.py:7: DeprecationWarning: Use of `minimize` with `x0.ndim != 1` is deprecated. Currently, singleton dimensions will be removed from `x0`, but an error will be raised in SciPy 1.11.0.
  result = minimize(loss_func, parameters_global, callback=callback)


3.4495413902868712
3.4495413902868712
3.4495413902868712


KeyboardInterrupt: 

In [ ]:
# # shape = qml.StronglyEntanglingLayers.shape(n_layers=n_layers, n_wires=n_wires)
# # params = rng.random(size=shape)
# # params = loaded_params
# # opt = qml.GradientDescentOptimizer() #TODO:// ADAM
# opt = qml.AdamOptimizer(0.05, beta1=0.9, beta2=0.999)
# print('starting plot:')
# plot_u_predict(parameters_global)
# for i in range(400):
#     print(i)
#     parameters_global, loss = opt.step_and_cost(loss_func, parameters_global)
#     if i%1==0:
#         print("Step ", i, "\n params = ", parameters_global,' loss = ',loss)
#         plot_u_predict(parameters_global)
    

        

In [ ]:
# print(params)
# np.save("optimized_params_sin_prime.npy", params)
# loaded_params = np.load("optimized_params.npy")
# print("Loaded parameters:", loaded_params)

In [ ]:
plot_u_predict(loaded_params)
sample_x = np.linspace(0, 1, 10, endpoint=True)
print(sample_x)
x=0.29
print(np.sin(x))
print(u_x(parameters=loaded_params,x=x))
